In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

MODEL

In [ ]:
class IrisClassifier(nn.Module):
    def __init__(self, hidden_size=8, output_size=3):
        super(IrisClassifier, self).__init__()
        self.fc1 = nn.Linear(4, hidden_size)  # 4 fitur input
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)


DATASET


In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder

class IrisDataset(Dataset):
    def __init__(self, csv_file_path):
        self.data = pd.read_csv(csv_file_path)
        self.le = LabelEncoder()
        self.data['label'] = self.le.fit_transform(self.data['species'])

        # Konversi ke float dan normalisasi
        fitur = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
        self.data[fitur] = self.data[fitur].apply(pd.to_numeric, errors='coerce')  # ubah ke float, error jadi NaN
        self.data = self.data.dropna(subset=fitur)  # buang baris yang error

        for col in fitur:
            self.data[col] = (self.data[col] - self.data[col].min()) / (self.data[col].max() - self.data[col].min())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        fitur = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
        x = torch.tensor(self.data.iloc[idx][fitur].values.astype('float32'))
        y = torch.tensor(self.data.iloc[idx]['label'], dtype=torch.long)
        return x, y


TRAIN

In [ ]:
from torch.utils.data import DataLoader, random_split

dataset = IrisDataset('/kaggle/input/iris-flower-dataset/IRIS.csv')
train_set, test_set = random_split(dataset, [120, 30])  # total 150

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
test_loader = DataLoader(test_set, batch_size=10)

model = IrisClassifier(8, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(30):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), 'iris_model.pth')
print("Model saved")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x, y in test_loader:
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

INFERENCE

In [ ]:
infer_df = pd.read_csv('/kaggle/input/iris-flower-dataset/IRIS.csv')
tb = infer_df.copy()
for col in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']:
    tb[col] = (tb[col] - tb[col].min()) / (tb[col].max() - tb[col].min())

x_infer = torch.tensor(tb[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values, dtype=torch.float32)

model = IrisClassifier(8, 3)
model.load_state_dict(torch.load('iris_model.pth'))
model.eval()

with torch.no_grad():
    logits = model(x_infer)
    preds = logits.argmax(dim=1)

le = LabelEncoder()
le.fit(infer_df['species'])  # fit label encoder ke nama spesies
predicted_labels = le.inverse_transform(preds.numpy())

infer_df['Prediksi_Spesies'] = predicted_labels
print(infer_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'Prediksi_Spesies']])

infer_df.to_csv("hasil_inference_iris.csv", index=False)
